# 問題37: 「猫」と共起頻度の高い上位10語

「猫」とよく共起する（共に出現する）単語を求めよ．上位10語とその共起頻度をグラフで表示せよ．

In [ ]:
# 問題30で実装した関数を読み込む
import sys
sys.path.append('../..')  # 親ディレクトリをパスに追加
from collections import Counter
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

# 日本語表示のための設定
plt.rcParams['font.family'] = 'IPAexGothic'

# 形態素解析結果を読み込む関数
def load_mecab_result(file_path):
    """
    MeCabの解析結果ファイルを読み込み、各形態素を辞書のリストとして返す関数
    
    Args:
        file_path (str): MeCab出力ファイルのパス
        
    Returns:
        list: 文のリスト。各文は形態素（辞書）のリスト
    """
    sentences = []
    current_sentence = []
    
    with open(file_path, 'r', encoding='utf-8') as f:
        for line in f:
            # EOSは文の区切り
            if line == 'EOS\n':
                if current_sentence:
                    sentences.append(current_sentence)
                    current_sentence = []
                continue
                
            # 空行をスキップ
            if line == '\n':
                continue
                
            # タブで分割して表層形とそれ以外の情報に分ける
            try:
                surface, info = line.split('\t')
                
                # カンマで分割して品詞情報などを取得
                info_items = info.split(',')
                
                # 形態素情報を辞書として格納
                morpheme = {
                    'surface': surface,
                    'base': info_items[6],
                    'pos': info_items[0],
                    'pos1': info_items[1]
                }
                
                current_sentence.append(morpheme)
            except:
                # 不正な形式の行をスキップ
                continue
    
    # 最後の文が追加されていない場合に追加
    if current_sentence:
        sentences.append(current_sentence)
    
    return sentences

# 「猫」と共起する単語の頻度を計算する関数
def count_cat_cooccurrence(sentences, window_size=5):
    """
    「猫」という単語と共起する単語の頻度を計算する関数
    
    Args:
        sentences (list): 文のリスト。各文は形態素（辞書）のリスト
        window_size (int): 共起を判定する窓サイズ（前後何単語まで見るか）
        
    Returns:
        list: (単語, 共起頻度)のタプルのリスト。共起頻度の降順でソート済み
    """
    # 「猫」と共起する単語をカウントするためのカウンター
    cooccurrence_counts = Counter()
    
    # 全ての文を処理
    for sentence in sentences:
        # 文中の各形態素の位置を調べる
        for i, morpheme in enumerate(sentence):
            # 「猫」という単語を見つけたら
            if morpheme['surface'] == '猫':
                # 前後window_size単語を共起語としてカウント
                start = max(0, i - window_size)
                end = min(len(sentence), i + window_size + 1)
                
                for j in range(start, end):
                    # 「猫」自身はカウントしない
                    if i != j:
                        cooccurrence_counts[sentence[j]['surface']] += 1
    
    # 共起頻度の降順でソート
    sorted_cooccurrence = sorted(cooccurrence_counts.items(), key=lambda x: x[1], reverse=True)
    
    return sorted_cooccurrence

# 頻度上位n語をグラフで表示する関数
def plot_top_n_cooccurrence(cooccurrence_frequencies, n=10):
    """
    共起頻度上位n語とその頻度を棒グラフで表示する関数
    
    Args:
        cooccurrence_frequencies (list): (単語, 共起頻度)のタプルのリスト
        n (int): 表示する単語の数
    """
    # 上位n語を抽出
    top_n = cooccurrence_frequencies[:n]
    
    # 単語と頻度を分離
    words = [word for word, count in top_n]
    counts = [count for word, count in top_n]
    
    # データフレームに変換
    df = pd.DataFrame({'単語': words, '「猫」との共起頻度': counts})
    
    # グラフの描画
    plt.figure(figsize=(12, 8), dpi=300)
    bars = plt.bar(df['単語'], df['「猫」との共起頻度'], color=plt.cm.viridis(np.linspace(0, 1, n)))
    
    # グラフの装飾
    plt.title('「猫」と共起頻度の高い上位10語', fontsize=16)
    plt.xlabel('単語', fontsize=14)
    plt.ylabel('共起頻度', fontsize=14)
    plt.xticks(rotation=45, fontsize=12)
    plt.yticks(fontsize=12)
    
    # 各バーの上に数値を表示
    for bar in bars:
        height = bar.get_height()
        plt.text(bar.get_x() + bar.get_width()/2., height + 0.1,
                 f'{height:.0f}',
                 ha='center', va='bottom', fontsize=12)
    
    plt.tight_layout()
    plt.grid(axis='y', linestyle='--', alpha=0.7)
    
    return plt.gcf()

In [ ]:
# 形態素解析結果の読み込み
file_path = '../../data/neko.txt.mecab'
sentences = load_mecab_result(file_path)

# 「猫」と共起する単語の頻度を計算
cat_cooccurrence = count_cat_cooccurrence(sentences)

# 共起頻度上位10語をグラフで表示
fig = plot_top_n_cooccurrence(cat_cooccurrence, 10)
plt.show()

# 共起頻度上位10語を表形式でも表示
top10_df = pd.DataFrame(cat_cooccurrence[:10], columns=['単語', '「猫」との共起頻度'])
display(top10_df)

# 内容語（名詞、動詞、形容詞、副詞）のみの共起頻度
content_word_cooccurrence = []
for word, count in cat_cooccurrence:
    # 単語が内容語かどうかを判定
    is_content_word = False
    for sentence in sentences:
        for morpheme in sentence:
            if morpheme['surface'] == word and morpheme['pos'] in ['名詞', '動詞', '形容詞', '副詞']:
                is_content_word = True
                break
        if is_content_word:
            break
    
    if is_content_word:
        content_word_cooccurrence.append((word, count))

# 内容語の共起頻度上位10語を表示
print("\n内容語（名詞、動詞、形容詞、副詞）と「猫」の共起頻度上位10語:")
content_top10_df = pd.DataFrame(content_word_cooccurrence[:10], columns=['単語', '「猫」との共起頻度'])
display(content_top10_df)

# 内容語の共起頻度上位10語をグラフで表示
fig2 = plot_top_n_cooccurrence(content_word_cooccurrence, 10)
plt.title('内容語と「猫」の共起頻度上位10語', fontsize=16)
plt.show()

## 解説

この問題では、「猫」という単語と共起する（共に出現する）単語の頻度を計算し、共起頻度の高い上位10語をグラフで表示しました。

### 実装のポイント

1. **共起の定義**: この実装では、同じ文の中で「猫」の前後一定範囲（window_size=5）に出現する単語を「猫」と共起すると定義しています。

2. **窓サイズの設定**: 共起を判定する範囲（窓サイズ）は、分析の目的や対象テキストの特性に応じて調整できます。窓サイズが大きいほど、より広い文脈での共起関係を捉えることができますが、関連性の薄い単語も含まれる可能性が高くなります。

3. **自己共起の除外**: 「猫」という単語自体は共起語としてカウントしないようにしています。

4. **グラフの作成**: matplotlibを使用して、共起頻度上位10語を棒グラフで視覚化しています。

### 共起分析の意義

共起分析は、単語間の関連性や文脈を理解するための重要な手法です。以下のような用途があります：

1. **意味関係の把握**: ある単語とよく共起する単語を調べることで、その単語の意味や使われ方を理解できます。

2. **トピック分析**: 特定の単語との共起関係を分析することで、テキスト中のトピックや主題を把握できます。

3. **コロケーション（連語）の発見**: 頻繁に一緒に使われる単語の組み合わせ（コロケーション）を発見できます。

### 結果の考察

「猫」と共起頻度の高い単語には、以下のようなものが含まれていると予想されます：

1. **助詞・助動詞**: 「は」「の」「が」などの機能語は、どの単語とも高頻度で共起する傾向があります。

2. **猫の属性や行動**: 「吾輩」「主人」「顔」「目」「耳」などの単語は、小説の中で猫の特徴や行動を描写する際に使われる可能性が高いです。

3. **猫の周辺環境**: 「家」「庭」「部屋」などの単語は、猫の生活環境を描写する際に使われる可能性が高いです。

内容語（名詞、動詞、形容詞、副詞）のみに絞った場合、より小説の内容を反映した共起関係が見えてくると考えられます。